# 1. Парсер vk

По каждому жанру нам нужно сделать 5 файлов для решения различных задач. Именно по этим файлам дальше будет разбита вся наша тетрадка. 

* Файл с подписчиками каждой группы жанра: 

```
    {"id группы 1" : [id подписчиков], .... }
```

* Файл с комментами каждой группы жанра: 

```
    {"id группы 1" : [комменты], .... }
```

Когда по каждому жанру будут скачаны подписчики, из низ надо сделать `set()` и по id каждого подписчика скачать: 


* Файл со стенками подписчиков: 

```
    {"подписчик 1" : [стена],  .... }
```

* Файл с топ-10 подписками по каждому подписчику: 

```
    {"подписчик 1" : [id его подписок],  .... }
```

Когда будут скачаны все паблики, из них надо сделать `set()` и по id каждого скачать: 

* Сотня последних постов с каждого паблика

```
    {"id паблика" :  [100 последних постов] , ....  }
```

In [1]:
# Загружаем необходимые библиотеки

import datetime                  # Пакет для работы с временными форматами
import pickle                    # Пакет для подгрузки данных специфического для питона формата
import requests                  # Пакет для скачки данных из этих ваших интернетов
import pandas as pd              # Пакет для работы с таблицами
import numpy as np               # Пакет для работы с векторами и матрицами
import matplotlib.pyplot as plt  # Пакет для строительства графиков
import time            # Пакет для работы со временем. Например, помогает ставить заглушки 
                       # time.sleep(секунды), необходимые для того что ВК не банил нащего
                       # сборщика данных из-за слишком частых запросов
        
# Пакет для красивых циклов. При желании его можно отключить. Тогда из всех циклов придётся 
# удалять команду tqdm_notebook.
from tqdm import tqdm_notebook   # подробнее: https://github.com/tqdm/tqdm

In [2]:
# мой номер странички
my_user_id = 91857120
# версия используемого API
version = '5.73' 
# подгружаем токен
with open('token.txt') as f:
    token = f.read()

In [3]:
def vkDownload(method, parameters, token=token, version=version):
    """
        Возвращает результат запроса по методу
        
        method: string
            метод из документации, который хотим использовать
            
        parameters: string
            параметры используемого метода
            
        token: string
            токен Oauth доступа
        
        version: string
            версия API
    """
    
    # составляем ссылку
    url = 'https://api.vk.com/method/{method}?{parameters}&access_token={token}&v={version}'
    url = url.format(method=method, parameters=parameters, token=token, version=version)
    # запрашиваем ссылку и переводим в json (словарь)
    response = requests.get(url).json()
    
    return response

vkDownload('users.get','user_ids=ppilif')

{'response': [{'first_name': 'Филипп',
   'id': 6045249,
   'last_name': 'Ульянкин'}]}

In [4]:
def makeBatch(uids, size=25):
    """
        Возвращает лист листов из пользователей, батчами по size человек 
        По умолчанию size=25
    """
    batches = [uids[i:i + size] for i in range(0,len(uids),size)]
    
    return batches 

## 1.1 Подписчики каждой группы для жанра

In [5]:
music_style = 'эстрада'

target_groups ={
     #'Лепс' : '52228413',
     'Пугачёва' : '24164800',
     #'Михайлов' : '925262',
     #'Меладзе' : '4336146',
     #'Ваенга' : '57465816',
     #'Киркоров' : '24344295',
     #'Валерия' : '17890994',
     #'Агутин' : '290390',
     #'Газманов' : '5640401',
     #'Басков' : '40705635',
     #'Николаев' : '106027463',
     'Ротару' : '37165678',
     'Лолита' : '1162252',
     'Леонтьев' : '9208',
     'Буйнов' : '157276422'
    }

In [6]:
# слоаврь группа-юзеры
group_users_dct = { }

Перед тем, как сохранять юзеров, надо профильтровать их на предмет существования... 

In [7]:
vkDownload('users.get','user_ids=barbizon_kid')

{'response': [{'deactivated': 'deleted',
   'first_name': 'Наталья',
   'id': 13034843,
   'last_name': 'Савостьянова'}]}

In [8]:
def checkExisting(uids):
    """
        Проверяет существование id и оставляет только не удаленные 
    """
    
    cur_inf = vkDownload('users.get','user_ids={}'.format(','.join(str(e) for e in uids)))
    cur_inf = cur_inf['response']
    clean_members = [us_inf['id'] for us_inf in cur_inf if us_inf.get('deactivated','real') == 'real']
    return clean_members


def getGroupMembers(group_id):
    """
        Возвращает список всех пользователей данной группы
        Итерации идут батчами по 1000 пользователей
        
        group_id: string
            идентификатор группы (ссылка)
    """
    
    # Узнаём число запросов, которое надо сделать 
    count = vkDownload('groups.getMembers','group_id=' + group_id)['response']['count']
    
    # выясняем, сколько запросов нам понадобиться
    n = int(np.ceil(count/1000))  
    
    # вектор, где мы будем хранить id пользователей
    members = []     
    
    for i in tqdm_notebook(range(n)): 
        # при помощи метода groups.getMembers получаем пользователей группы
        current_members = vkDownload('groups.getMembers','group_id='+group_id+'&offset='+str(1000*i))
        current_members = current_members['response']['items']
        
        # проверим реально ли существуют все юзеры из списка выше
        for i in range(0,1000,200):
            time.sleep(0.3)
            members.extend(checkExisting(current_members[i:i+200]))
        
        # перед следующим запросом немножко подождем
        time.sleep(0.3)
        
    return members

In [9]:
# по каждому id группы сохраняем подписчиков 
for k,v in tqdm_notebook(target_groups.items()):
    members = getGroupMembers(target_groups[k]) 
    print(k,':',len(members))
    group_users_dct[k] = members

HBox(children=(IntProgress(value=0, max=5), HTML(value='')))

HBox(children=(IntProgress(value=0, max=33), HTML(value='')))

Пугачёва : 29177


HBox(children=(IntProgress(value=0, max=15), HTML(value='')))

Ротару : 12736


...

Лолита : 26313


HBox(children=(IntProgress(value=0, max=9), HTML(value='')))

Леонтьев : 7740


HBox(children=(IntProgress(value=0, max=2), HTML(value='')))

Буйнов : 547



In [11]:
group_users_dct['Пугачёва'][-10:]

[521772661,
 521887515,
 521923792,
 522548600,
 522998936,
 523271589,
 523478186,
 523508174,
 523729785,
 523830262]

In [12]:
# Сохраняем словарик для текущего жанра 
with open('../data/{}_group_users'.format(music_style), 'wb') as f:
    pickle.dump(group_users_dct, f)

## 1.2 Файл с комментами каждой группы жанра

Скачиваем с группы все посты.

In [13]:
def getPosts(group_id):
    """
        Качает по группе список всех постов, которые в 
        ней есть и возвращает список из их id 
    """
    
    post_ids = [ ] # для сбора id
    
    # Выясним сколько всего в группе постов 
    n = vkDownload('wall.get','owner_id=-{}'.format(group_id))['response']['count']
    print("В группе {} должно быть {} постов".format(group_id,n))
    
    for i in tqdm_notebook(range(0, n + 100, 100)):
        time.sleep(0.3)
        
        wall = vkDownload('wall.get','owner_id=-{}&count=100&offset='.format(group_id)+str(i))['response']['items']
        # если захочется отделить комменты группы от других, есть поле from_id 
        post_ids.extend([item['id'] for item in wall])
        
    return post_ids


def prepareComments(group_id, posts):
    """
        Мы будем скачивать 25 запросами за раз, эта функция готовит запросы 
        к последущему использованию, понимая сколько комментов под каким постом, 
        выкидывая посты с нулевым количеством комментов и делая тройки (группа, пост, оффсет)
    """

    triple_comments = [ ] # для троек (group_id, post_id, offset)

    # делаем из всех постов батчи по 25 штук, чтобы было быстрее
    posts_batch = makeBatch(posts)

    for batch in tqdm_notebook(posts_batch):
        time.sleep(0.4)

        # готовим запрос для комметов из батча, который выяснит число комментов под постом
        begin = 'https://api.vk.com/method/execute?code=return['
        end = '];&access_token='+token+'&v=5.78'
        middle = ''

        # для каждого поста из батча используем метод wall.getComments
        for bt in batch:            
            middle += 'API.wall.getComments({"owner_id":"-' + str(group_id) + '","post_id":"' + str(bt) + '"}),'

        # делаем запрос для текущих 25 комментов
        requests.get(begin + middle[:-1] + end)

        # выясняем сколько всего комментов под постом 
        s = requests.get(begin + middle[:-1] + end)

        # СТРАННОСТЬ: 
        # В каждом запросе нужные мне 25 ответов в поле 'response' 
        # и 26 сообщений Permission to perform this action is denied  в поле 'execute_errors'
        # Но но но я же могу его сделать...  Какие нафиг запреты? Мб скоро прикроют API - ? 

        # по добытой информации готовим запросы на будущее

        for post,item in zip(batch, s.json()['response']):
            if item is False:
                print('False в посте {} группы {}'.format(post, group_id))
                continue

            n = item['count']

            # пост с нулевым числом комментов игнорим
            if n == 0:
                continue 
            else:
                # для каждой сотни комментов из-под поста с комментами будет свой запрос для добычи
                for i in range(0, n, 100):
                    triple_comments.append((group_id, post, i))
    return triple_comments


def getComments(triple_comments):
    """
        Делит элементы вектора вида (group_id, post_id, offset), 
        на батчи по 15 штук и качает комменты аки демон
    """
    
    comments = [ ] # для сбора комментов 

    # делаем батчи 
    precom_batch = makeBatch(triple_comments, 15)

    for batch in tqdm_notebook(precom_batch):
        time.sleep(0.4)
        # клепаем запрос 
        begin = 'https://api.vk.com/method/execute?code=return['
        end = '];&access_token='+token+'&v=5.78'
        middle = ''

        for bt in batch:            
            middle += 'API.wall.getComments({"owner_id":"-' + str(bt[0]) + '","post_id":"' + str(bt[1]) + '",\
            "need_likes":"1","preview_length":"0","count":"100","offset":"'+ str(bt[2])+'"}),'

        # делаем запрос для текущих 20 комментов
        s = requests.get(begin + middle[:-1] + end)

        # распаковываем добытое добро
        cur_com = [ ]
        for item, post in zip(s.json()['response'], batch):
            for jtem in item['items']:
                jtem['likes'] = jtem['likes']['count']
                jtem['post_id'] = post[1]
                jtem['group_id'] = post[0]
                cur_com.append(jtem)
                
        comments.extend(cur_com) 
        
    return comments    

In [14]:
unique_comments = { }

for name in target_groups:
    
    # качаем посты 
    group_id = target_groups[name]
    posts = getPosts(group_id)

    print('Из группы {} имени {} скачалось {} постов'.format(target_groups[name], name, len(posts)))
    
    # готовим батчи из запросов для комментов
    triple_comments = prepareComments(group_id, posts)
    
    print('Число запросов: {}'.format(len(triple_comments)))
    
    # качаем комменты
    comments = getComments(triple_comments)
    
    setlen = len(set([str(comment['post_id']) + '_' + str(comment['id']) for comment in comments]))
    print('Число комментов: {}, из них уникальных: {}'.format(len(comments), setlen ))
    
    # смотрим на сотню последних комментов 
    for comment in comments[-100:]:
        print(comment['post_id'],comment['id'])
        print(comment['text'])
        print('--------------------')
        
    # записываем накачаное добро в словарик
    unique_comments[name] = comments
    
    print('====================================================================')
    

В группе 24164800 17024 постов


HBox(children=(IntProgress(value=0, max=172), HTML(value='')))


Число постов в группе Пугачёва: 17024


HBox(children=(IntProgress(value=0, max=681), HTML(value='')))

KeyboardInterrupt: 

In [ ]:
'../data/{}_group_comments'.format(music_style)

In [ ]:
# Сохраняем ну хоть что-нибудь... 
with open('../data/{}_group_comments'.format(music_style), 'wb') as f:
    pickle.dump(group_users_dct, f)

In [ ]:
!ls ../data

## 1.3 Файл с топ-10 подписками по каждому подписчику

In [ ]:
unique_users = group_users_dct['24164800'][:100]
unique_users[:5]

In [ ]:
def makePublicUrl(batch, token=token):
    """
        Создаёт ссылку для метода execute на основе батчей пользователей для скачки пабликов
            batch - батч из id 
            token - ключ доступа
    """
    
    # составляем ссылку из кусочков
    begin = 'https://api.vk.com/method/execute?code=return['
    end = '];&access_token='+token+'&v=5.78'
    middle = ''
    # для каждого пользователя из батча используем метод wall.get 
    for bt in batch:
        middle += 'API.users.getSubscriptions({"user_id": "'+str(bt)+'"}),'   
    return begin + middle[:-1] + end 


def getUsersPublics(uids):
    """
    Достаёт список пабликов для всех людей из списка uids
        uids - id людей 
    """
    
    batches = makeBatch(uids) # дробим на батчи
    
    # сюда будем записывать соответсвие между юзером и пабликами 
    id_group_list = { }
    
    for item in tqdm_notebook(batches):
        time.sleep(0.3)
        response = requests.get(makePublicUrl(item))
        cur_resp = response.json()['response']
        
        # распаковываем список подписок по подписчикам
        for uid,cur in zip(item,cur_resp):
            id_group_list[uid] = cur['groups']['items'][:10] # первые 10
            
    return(id_group_list)

In [ ]:
users_top_publics = getUsersPublics(unique_users[:2])

In [ ]:
users_top_publics

In [ ]:
# Сохраняем это чудо 

## 1.4 Файл со стенками подписчиков 

* Додумываем процедуру сбора

In [ ]:
# !!! Внимание! По этому методу есть лимит в 10к запросов. Имеет смысл брать жанр, 
# после делать по нему случайную выборку в 15к людей и качать стенки только по ним !!!

In [ ]:
def makeWallUrl(batch, token=token):
    """
        Создаёт ссылку для метода execute на основе батчей пользователей
    """
    # составляем ссылку из кусочков
    begin = 'https://api.vk.com/method/execute?code=return['
    end = '];&access_token='+token+'&v='+version
    middle = ''
    # для каждого пользователя из батча используем метод wall.get 
    for bt in batch:
        middle += 'API.wall.get({"owner_id": "'+str(bt)+'","count":"100"}),'
        
    return begin + middle[:-1] + end 

def getAllPosts(candidate):
    """
        Возвращает список словарей, в каждом их которых посты пользователей
    """
    
    # инициализируем список для хранения постов
    wall = []
    # берем всех людей текущего кандидата
    people = candidate_people[candidate]
    
    # рандомно перемешаем наших пользователей, чтобы обеспечить репрезентативность выборки
    np.random.shuffle(people)
    
    # создадим свой небольшой класс для кастомной ошибки 
    # на случай превышения лимита запросов
    class LimitExceeded(Exception):
        pass
    
    # батчи пользователей
    batches = makeBatch(people)
    
    # итерируем по батчам
    for item in tqdm_notebook(batches):
        # если насобирали 10 тысяч пользователей, останавливаемся
        if len(wall) < 10000:
            try:
                # пробуем выгрузить по батчу стенки пользователей
                res = requests.get(makeWallUrl(item))
                res = res.json()['response']
                
                # если нас забанили (все респонсы пусты), поднимем ошибку
                if res == [False]*25:
                    raise LimitExceeded("Превышено число запросов")
                
                # записываем полученные данные в список
                wall.extend(res)
                
            except LimitExceeded:
                # если забанили - подождем минутку, чтобы обновились лимиты запросов
                time.sleep(60)
                
            # в конце каждой итерации ждем 2 секунды, чтобы не словить бан
            time.sleep(2)
        else:
            break
            
    return wall

## 1.5 Сотня последних постов с каждого паблика

Мы скачали все топы подписок по каждому подписчику. Теперь нужно сделать по ним `set()`.

In [ ]:
# Тут появится код в самом-самом конце :3 

In [ ]:
# Сделать по аналогии для 100 постов с каждого паблика !!!

def makePublicUrl(batch, token=token):
    """
        Создаёт ссылку для метода execute на основе батчей пользователей для скачки пабликов
            batch - батч из id 
            token - ключ доступа
    """
    
    # составляем ссылку из кусочков
    begin = 'https://api.vk.com/method/execute?code=return['
    end = '];&access_token='+token+'&v=5.78'
    middle = ''
    # для каждого пользователя из батча используем метод wall.get 
    for bt in batch:
        middle += 'API.users.getSubscriptions({"user_id": "'+str(bt)+'"}),'   
    return begin + middle[:-1] + end 

def getUsersPublics(uids):
    """
    Достаёт список пабликов для всех людей из списка uids
        uids - id людей 
    """
    
    batches = makeBatch(uids) # дробим на батчи
    
    # сюда будем записывать соответсвие между юзером и пабликами 
    id_group_list = { }
    
    for item in tqdm_notebook(batches):
        time.sleep(0.3)
        response = requests.get(makePublicUrl(item))
        cur_resp = response.json()['response']
        
        # распаковываем список подписок по подписчикам
        for uid,cur in zip(item,cur_resp):
            id_group_list[uid] = cur['groups']['items'][:10] # первые 10
            
    return(id_group_list)